In [4]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
#%autosave 180
import matplotlib.pyplot as plt
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np

In [22]:
# LOAD the first 10,000 PREDICTIONS/CONFIDENCE MAPS THAT COME OUT OF DEEPLABCUT for a particular video
# Notes: 
#        Equivalent to 10000/25fps = 400seconds
#        Dimensionts: [# of Frames, width, height]
#        DLC PREDICTS ON 1280 x 1024 images, but they are downsampled to 128 x 160
#               i.e. if we need higher resolution, it can be obtained easily

predictions = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/predictions_10k_new_surfaces.npy')
print (predictions.shape)

max_frames = 10000

predictions=predictions[:max_frames]
print (predictions.shape)

# im = plt.imshow(predictions[0])
# plt.colorbar()
# plt.show()

(10000, 128, 160)
(10000, 128, 160)


In [24]:
# REPROCESS HEATMAPS
for k in range(predictions.shape[0]):
    predictions[k]=(predictions[k]-np.min(predictions[k]))/(np.max(predictions[k]-np.min(predictions[k])))

threshold = .7
idx = np.where(predictions<threshold)
predictions[idx]=0

for k in range(predictions.shape[0]):
    predictions[k]=(predictions[k]-np.min(predictions[k]))/(np.max(predictions[k]-np.min(predictions[k])))

print (np.max(predictions[0]), np.min(predictions[0]))

1.0 0.0


In [41]:
print (np.max(images[100]), np.min(images[100]))

1.0 0.01568627450980392


In [25]:
# np.save('/home/cat/prediction0.npy', predictions[0])
# np.save('/home/cat/images0.npy',images[0])


In [26]:
# LOAD THE FIRST 1,000 VIDEO FRAMES (there are 10k in the saved file)

import cv2
vidcap = cv2.VideoCapture('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi')
images = np.zeros((predictions.shape[0],
                  predictions.shape[1],
                  predictions.shape[2],
                  3))
count=0
max_count = max_frames # THERE ARE 10,000 frames here;
while True:
    success, image = vidcap.read()
    images[count]=image[::8,::8]
    if count%100==0:
        print (count)
    count += 1
    if count>=max_count:
        break

images = np.transpose(np.array(images),(0,-1,1,2))
images = images/255.

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [27]:
# # VISUALIZE AS SANITY CHECK SIDE BY SIDE
# # SELECT FRAME

# frame = 50

# ax=plt.subplot(2,2,1)
# plt.title("Raw video frame",fontsize=16)
# #print (images[frame].shape)
# plt.imshow(images[frame].sum(0),alpha=1)
# ax=plt.subplot(2,2,2)
# plt.title("DLC (sum) heatmaps",fontsize=16)
# plt.imshow(predictions[frame].squeeze(),alpha=1)
# ax=plt.subplot(2,2,3)
# plt.title("Overlayed as a sanity check",fontsize=16)
# plt.imshow(images[frame].sum(0),alpha=.5)
# plt.imshow(predictions[frame].squeeze(),alpha=.5)
# plt.show()


In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.utils.data
import torch.optim as optim

In [29]:
class Cnndnn(nn.Module):
    def __init__(self,hdim=48):
        super(Cnndnn, self).__init__()
        
        #input channels, output channels, kernel, stride
        self.conv1 = nn.Conv2d(3,32,3,1)
        self.conv2 = nn.Conv2d(32,64,3,1)
        self.conv3 = nn.Conv2d(64,128,3,1)
        self.conv4 = nn.Conv2d(128,256,3,1)
        
        self.deconv4 = nn.ConvTranspose2d(256,128,3,1)
        self.deconv3 = nn.ConvTranspose2d(128,64,3,1)
        self.deconv2 = nn.ConvTranspose2d(64,32,3,1)
        self.deconv1 = nn.ConvTranspose2d(32,1,3,1)
        
        self.linearconv = nn.Linear(12288,hdim)
        self.lineardeconv = nn.Linear(hdim,12288)
        
        self.pool = nn.MaxPool2d(2, return_indices=True)
        self.unpool = nn.MaxUnpool2d(2)
        
        
    def forward(self,x):
        
        #pass through generic CNN
        x = self.conv1(x)
        x = F.relu(x)
        x,idx1 = self.pool(x)
        
        x = self.conv2(x)
        x = F.relu(x)
        x,idx2 = self.pool(x)
        
        x = self.conv3(x)
        x = F.relu(x)
        x,idx3 = self.pool(x)
        
        x = self.conv4(x)
        x = F.relu(x)
        x,idx4 = self.pool(x)
        
        #transform cnn output to the 'hidden'/encoding layer thing
        x = self.linearconv(torch.flatten(x,1))
        
        #do rnn things here if desired:
        # x = rnn(x,xprev)
        
        #transform back to 'feature'-space
        x = self.lineardeconv(x)
        x = x.view(-1,256,6,8)

        #blow back out with deconvNN
        x = self.unpool(x,idx4,output_size=(12,16))
        x = F.relu(x)
        x = self.deconv4(x)

        x = self.unpool(x,idx3,output_size=(28,36))
        x = F.relu(x)
        x = self.deconv3(x)
        
        x = self.unpool(x,idx2,output_size=(61,77))
        x = F.relu(x)
        x = self.deconv2(x)
        
        x = self.unpool(x,idx1,output_size=(126,158))
        x = F.relu(x)
        x = self.deconv1(x)
        
        x = torch.sigmoid(x)
        
        return x

cnndnn = Cnndnn().to(device='cuda')

In [30]:
#foo = cnndnn(torch.rand(10,3,128,160))

In [31]:
#img_data_np = np.random.rand(100,3,128,160)

In [32]:
#heatmaps_np = np.random.rand(100,3,128,160)

In [33]:
# #predictions = predictions [:100]
# print (predictions.shape)
# print (images.shape)



In [34]:
#dataset = torch.utils.data.TensorDataset(torch.tensor(img_data_np).float(),torch.tensor(heatmaps_np).float())
dataset = torch.utils.data.TensorDataset(torch.tensor(images).float(),torch.tensor(predictions).float())
data_loader = torch.utils.data.DataLoader(dataset,batch_size=10,shuffle=True)

In [38]:
# STOCHASTIC GD
# optimizer = optim.SGD(cnndnn.parameters(), lr=0.01, momentum=0.9)

# SECOND OPTION
optimizer = optim.Adam(cnndnn.parameters(), lr=0.01)

In [39]:
loss_list = []
for i,data in enumerate(data_loader):
    
    inputs,targets = data

    inputs = inputs.to(device='cuda')
    targets = targets.to(device='cuda')
    
    optimizer.zero_grad()
    
    output = cnndnn(inputs)
    
    #loss = ((output-targets)**2).sum()
    loss = ((output-targets)**2).mean()
    loss_list.append(loss.cpu().detach().numpy().squeeze())
    
    loss.backward()
    optimizer.step()
    
    print('iter {}, loss: {}'.format(i,loss.item()))

iter 0, loss: 0.016355004161596298
iter 1, loss: 0.014792589470744133
iter 2, loss: 0.014591893181204796
iter 3, loss: 0.012872978113591671
iter 4, loss: 0.01579000987112522
iter 5, loss: 0.013936401344835758
iter 6, loss: 0.015597126446664333
iter 7, loss: 0.014099931344389915
iter 8, loss: 0.01513295341283083
iter 9, loss: 0.016701053828001022
iter 10, loss: 0.016882119700312614
iter 11, loss: 0.017583508044481277
iter 12, loss: 0.01598624885082245
iter 13, loss: 0.014567789621651173
iter 14, loss: 0.015388421714305878
iter 15, loss: 0.018162574619054794
iter 16, loss: 0.01653827354311943
iter 17, loss: 0.017816444858908653
iter 18, loss: 0.019425617530941963
iter 19, loss: 0.015474935062229633
iter 20, loss: 0.01564696989953518
iter 21, loss: 0.01646873913705349
iter 22, loss: 0.013868691399693489
iter 23, loss: 0.015776963904500008
iter 24, loss: 0.014495806768536568
iter 25, loss: 0.014976256527006626
iter 26, loss: 0.015755033120512962
iter 27, loss: 0.015502966940402985
iter 28,

iter 230, loss: 0.010235314257442951
iter 231, loss: 0.011037880554795265
iter 232, loss: 0.011578169651329517
iter 233, loss: 0.010492886416614056
iter 234, loss: 0.011315676383674145
iter 235, loss: 0.009558012709021568
iter 236, loss: 0.010106232948601246
iter 237, loss: 0.008276681415736675
iter 238, loss: 0.010426374152302742
iter 239, loss: 0.010992367751896381
iter 240, loss: 0.011690828949213028
iter 241, loss: 0.008822466246783733
iter 242, loss: 0.011453337036073208
iter 243, loss: 0.012088739313185215
iter 244, loss: 0.010057464241981506
iter 245, loss: 0.009533955715596676
iter 246, loss: 0.011645735241472721
iter 247, loss: 0.010203249752521515
iter 248, loss: 0.009877406060695648
iter 249, loss: 0.009769875556230545
iter 250, loss: 0.01088436134159565
iter 251, loss: 0.010709303431212902
iter 252, loss: 0.011789735406637192
iter 253, loss: 0.009813912212848663
iter 254, loss: 0.010265651158988476
iter 255, loss: 0.009775716811418533
iter 256, loss: 0.013012335635721684
it

iter 457, loss: 0.012454205192625523
iter 458, loss: 0.007970437407493591
iter 459, loss: 0.011106523685157299
iter 460, loss: 0.01256287470459938
iter 461, loss: 0.011322543025016785
iter 462, loss: 0.01086342241615057
iter 463, loss: 0.012018950656056404
iter 464, loss: 0.01095682941377163
iter 465, loss: 0.01242367085069418
iter 466, loss: 0.011477169580757618
iter 467, loss: 0.010068594478070736
iter 468, loss: 0.010713291354477406
iter 469, loss: 0.010602173395454884
iter 470, loss: 0.009767798706889153
iter 471, loss: 0.010943601839244366
iter 472, loss: 0.009700260125100613
iter 473, loss: 0.01156742125749588
iter 474, loss: 0.011418755166232586
iter 475, loss: 0.011122776195406914
iter 476, loss: 0.009572319686412811
iter 477, loss: 0.010822320356965065
iter 478, loss: 0.011174860410392284
iter 479, loss: 0.011113710701465607
iter 480, loss: 0.008369590155780315
iter 481, loss: 0.010362706147134304
iter 482, loss: 0.009596926160156727
iter 483, loss: 0.011628607288002968
iter 4

iter 680, loss: 0.010552250780165195
iter 681, loss: 0.010261359624564648
iter 682, loss: 0.011818783357739449
iter 683, loss: 0.008494986221194267
iter 684, loss: 0.010940895415842533
iter 685, loss: 0.009685730561614037
iter 686, loss: 0.01123658288270235
iter 687, loss: 0.010519412346184254
iter 688, loss: 0.010739484801888466
iter 689, loss: 0.011116940528154373
iter 690, loss: 0.01002185232937336
iter 691, loss: 0.01056078914552927
iter 692, loss: 0.010244389064610004
iter 693, loss: 0.009769368916749954
iter 694, loss: 0.010264530777931213
iter 695, loss: 0.011528811417520046
iter 696, loss: 0.009588856250047684
iter 697, loss: 0.00944112055003643
iter 698, loss: 0.011518394574522972
iter 699, loss: 0.009207692928612232
iter 700, loss: 0.011063316836953163
iter 701, loss: 0.010829199105501175
iter 702, loss: 0.009332781657576561
iter 703, loss: 0.011237447150051594
iter 704, loss: 0.010370845906436443
iter 705, loss: 0.010074877180159092
iter 706, loss: 0.009252602234482765
iter 

iter 907, loss: 0.011074579320847988
iter 908, loss: 0.010780352167785168
iter 909, loss: 0.009055730886757374
iter 910, loss: 0.011088979430496693
iter 911, loss: 0.0102769760414958
iter 912, loss: 0.010726856999099255
iter 913, loss: 0.010441903956234455
iter 914, loss: 0.010847986675798893
iter 915, loss: 0.011796323582530022
iter 916, loss: 0.01128648966550827
iter 917, loss: 0.010189377702772617
iter 918, loss: 0.011734587140381336
iter 919, loss: 0.010038948617875576
iter 920, loss: 0.010363350622355938
iter 921, loss: 0.0105472132563591
iter 922, loss: 0.009238515049219131
iter 923, loss: 0.008667541667819023
iter 924, loss: 0.0095879677683115
iter 925, loss: 0.010199366137385368
iter 926, loss: 0.01177337858825922
iter 927, loss: 0.012055839411914349
iter 928, loss: 0.010370470583438873
iter 929, loss: 0.010325070470571518
iter 930, loss: 0.011108648963272572
iter 931, loss: 0.008901610039174557
iter 932, loss: 0.01036106888204813
iter 933, loss: 0.011012990027666092
iter 934, 

In [43]:
# VISUALIZE LAST PREDICTED OUTPUT
frame_id = 33

# SELECT IMAGE
raw = images[frame_id].transpose(1,2,0)
print (" raw: ", raw.shape)

ax=plt.subplot(2,2,1)
plt.title("Input")
plt.imshow(raw)

# TARGET
ax=plt.subplot(2,2,2)
plt.title("target")
print (targets.shape)
target1 = predictions[frame_id]
plt.imshow(target1)


# PREDICTION
ax=plt.subplot(2,2,3)
plt.title("prediction")

frame_selected = torch.from_numpy(raw.transpose(2,0,1)).float()[None].to(device='cuda')
print ("frame to be predicted: ", frame_selected.shape)
predicted = cnndnn(frame_selected).cpu().detach().numpy().squeeze()
print (predicted.shape)
plt.imshow(predicted)

# LOSS
ax=plt.subplot(2,2,4)
plt.title("loss")
plt.plot(loss_list)
plt.xlim(0,len(loss_list))


plt.suptitle("Predicting frame; "+str(frame_id))
plt.show()


 raw:  (128, 160, 3)
torch.Size([10, 128, 160])
frame to be predicted:  torch.Size([1, 3, 128, 160])
(128, 160)


In [19]:
# REPREDICT ON A SELECTED FRAME
frame_id = 0

#print (images.shape)
frame_selected = torch.from_numpy(images[frame_id]).float()[None]
print ("frame selected: ", frame_selected.shape, type(frame_selected[0][0][0][0]))

output = cnndnn(frame_selected)


raw = inputs[0].cpu().detach().numpy().transpose(1,2,0)
print (raw.shape)

ax=plt.subplot(2,1,1)
plt.imshow(raw)


# 
ax=plt.subplot(2,1,2)
predicted = output[0].cpu().detach().numpy().transpose(1,2,0)
plt.imshow(predicted)
plt.show()

frame selected:  torch.Size([1, 3, 128, 160]) <class 'torch.Tensor'>
(128, 160, 3)
